In [1]:
import pandas as pd
import os
import glob

In [2]:
# Working directory
project_dir = '/zfs/projects/darc/wolee_edehaan_suzienoh-exploratory-ml'
os.chdir(project_dir)

In [3]:
def get_prediction_file_paths(directory):
    # Use glob to get all csv files in the directory
    csv_files = glob.glob(os.path.join(directory, '*prediction*.csv'))
    return csv_files

In [4]:
def postprocess_predictions(prediction_df, prediction_col='pred', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [5]:
def maximum_return(prediction_df, prediction_col='ret', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [6]:
def create_result(prediction_parent_path, result_file_name=None, period='month', max_return=False):
    # Get the prediction data paths
    prediction_data_paths = get_prediction_file_paths(prediction_parent_path)
    
    # Postprocess the prediction and append all the results together
    results = pd.DataFrame()
    for df_path in prediction_data_paths:
        df = pd.read_csv(df_path)
        if max_return:
            if period=='month':
                year_vret = maximum_return(df, prediction_col='ret', period='month')
            elif period=='quarter':
                year_vret = maximum_return(df, prediction_col='retq', period='quarter')
            else:
                print('Please input period as either month or quarter')
        else:
            year_vret = postprocess_predictions(df, period=period)
        results = pd.concat([results, year_vret]).reset_index(drop=True)
    
    # Sort the results
    sorted_results = results.sort_values(by=['date', 'rank'],  ascending=[True, True]).reset_index(drop=True)
    
    # Save the sorted results to the same parent directory if file name is given
    if result_file_name:
        sorted_results.to_csv(f'{prediction_parent_path}/{result_file_name}', index=False)
        
    return sorted_results

In [7]:
def compute_returns(path):
    
    # Load data
    df = pd.read_csv(path)

    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

In [8]:
def compute_returns_df(df):
    
    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

### Current Quarterly New Restricted

In [9]:
period = 'quarter'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_restricted'
result_file_name = 'result.csv'

In [10]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [11]:
compute_returns_df(results)

/tmp/ipykernel_115797/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.013289
1,1.0,0.025109
2,2.0,0.029242
3,3.0,0.031388
4,4.0,0.033673
5,5.0,0.030429
6,6.0,0.034697
7,7.0,0.034552
8,8.0,0.046154
9,9.0,0.054191


### Current Quarterly New Vars

In [12]:
period = 'quarter'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_vars'
result_file_name = 'result.csv'

In [13]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [14]:
compute_returns_df(results)

/tmp/ipykernel_115797/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.027730
1,1.0,0.025904
2,2.0,0.030771
3,3.0,0.030043
4,4.0,0.031727
5,5.0,0.035962
6,6.0,0.034297
7,7.0,0.031349
8,8.0,0.033724
9,9.0,0.032827


### Current Monthly New Restricted

In [15]:
period = 'month'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_restricted'
result_file_name = 'result.csv'

In [16]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [17]:
compute_returns_df(results)

/tmp/ipykernel_115797/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,-0.003345
1,1.0,0.004338
2,2.0,0.007972
3,3.0,0.009160
4,4.0,0.010232
5,5.0,0.012578
6,6.0,0.013128
7,7.0,0.014217
8,8.0,0.015865
9,9.0,0.021817


### Current Monthly New Vars

In [18]:
period = 'month'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_vars'
result_file_name = 'result.csv'

In [19]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [20]:
compute_returns_df(results)

/tmp/ipykernel_115797/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.005073
1,1.0,0.008334
2,2.0,0.010095
3,3.0,0.010380
4,4.0,0.011332
5,5.0,0.013042
6,6.0,0.011259
7,7.0,0.011464
8,8.0,0.012775
9,9.0,0.014816


### Monthly Old Vars

In [21]:
compute_returns('kevin/output/prediction/monthly_old_vars/result.csv')

/tmp/ipykernel_115797/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.003681
1,1.0,0.007112
2,2.0,0.009797
3,3.0,0.009424
4,4.0,0.011221
5,5.0,0.011056
6,6.0,0.011664
7,7.0,0.010712
8,8.0,0.012864
9,9.0,0.017309


### Analyzing result files

In [22]:
def analyze_result_files(result_dir):
    result = pd.read_csv(result_dir)
    result['year'] = result['date'].str[:4]
    summary = pd.pivot_table(data=result, index='year', columns='rank', values='port_ret', aggfunc='mean')
    summary['return'] = summary[9.0] - summary[0.0]

    return summary

In [23]:
quarterly_restricted_result_dir = 'kevin/output/prediction/quarterly_new_restricted/result.csv'
quarterly_new_result_dir = 'kevin/output/prediction/quarterly_new_vars/result.csv'
monthly_restricted_result_dir = 'kevin/output/prediction/monthly_new_restricted/result.csv'
monthly_new_result_dir = 'kevin/output/prediction/monthly_new_vars/result.csv'

### Quarterly New Restricted

In [24]:
analyze_result_files(quarterly_restricted_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.053303,0.081361,0.072383,0.075900,0.089916,0.076854,0.078322,0.075168,0.083178,0.070111,0.016807
1986,0.038675,0.049798,0.034040,0.043670,0.044261,0.036239,0.051023,0.034492,0.044243,0.041298,0.002623
1987,0.009976,0.030064,0.011052,0.029276,0.023999,0.016639,0.003107,0.024213,0.016547,0.014126,0.004150
1988,-0.017456,0.026807,0.046970,0.027694,0.048664,0.040783,0.044029,0.039901,0.030149,0.047634,0.065090
1989,0.055127,0.058661,0.059039,0.058313,0.054836,0.072495,0.064798,0.088405,0.064325,0.070516,0.015389
1990,-0.095201,-0.058611,-0.023418,-0.003278,-0.007505,0.009274,-0.006832,0.025546,0.078994,0.121227,0.216428
1991,0.053497,0.109679,0.104737,0.071752,0.074007,0.056903,0.080208,0.081509,0.072164,0.084542,0.031045
1992,0.013868,0.024895,0.019647,0.008368,0.045891,0.037029,0.047281,0.007833,0.019182,0.011895,-0.001972
1993,0.030373,0.005591,0.040451,0.022601,0.030861,0.016112,0.026673,0.020998,0.037648,0.051395,0.021021


### Quarterly New Vars

In [25]:
analyze_result_files(quarterly_new_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.030180,0.061176,0.069046,0.071347,0.076493,0.079774,0.070543,0.073199,0.089351,0.079883,0.049703
1986,0.042264,0.049870,0.048696,0.044210,0.049211,0.049712,0.012023,0.037079,0.051382,0.039416,-0.002848
1987,0.021649,0.014500,0.027343,0.013840,0.015754,0.014581,0.032596,0.008856,0.026744,0.021388,-0.000261
1988,0.011738,0.034581,0.038234,0.047962,0.044277,0.036498,0.036595,0.048742,0.053234,0.042540,0.030802
1989,0.083657,0.050571,0.049826,0.085821,0.072637,0.064149,0.071560,0.072194,0.059130,0.036253,-0.047404
1990,-0.058608,-0.051252,-0.041467,-0.039402,-0.010268,-0.006094,-0.019812,0.001104,0.010601,0.006009,0.064617
1991,0.075199,0.081725,0.091096,0.074522,0.079104,0.089907,0.080799,0.078698,0.074255,0.125870,0.050671
1992,0.015363,0.025312,0.023947,0.021986,0.044713,0.048624,0.044637,0.023076,0.056121,0.006212,-0.009151
1993,0.043929,0.030823,0.035990,0.026535,0.028488,0.035987,0.031055,0.025984,0.013625,0.022419,-0.021510


### Monthly New Restricted

In [26]:
analyze_result_files(monthly_restricted_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.007782,0.011813,0.019460,0.017572,0.021380,0.022893,0.023536,0.029384,0.026642,0.025835,0.018053
1986,0.014574,0.011960,0.018578,0.007464,0.017883,0.009836,0.012055,0.012024,0.016417,0.015458,0.000885
1987,0.011090,0.010322,0.006674,0.005815,0.003624,0.000122,0.004672,-0.003979,0.003178,0.003325,-0.007765
1988,0.006600,0.011435,0.014111,0.013456,0.012173,0.013616,0.015319,0.013950,0.018627,0.020710,0.014110
1989,0.012712,0.019225,0.023757,0.018878,0.023465,0.020875,0.024537,0.021647,0.022032,0.027609,0.014897
1990,-0.035582,-0.007281,-0.000781,-0.000133,-0.011434,0.001176,-0.002714,0.000332,0.000488,0.000400,0.035982
1991,0.023212,0.035030,0.030159,0.026895,0.023354,0.021917,0.025314,0.025098,0.028362,0.026614,0.003401
1992,0.000234,0.007593,0.007893,0.009300,0.012536,0.009336,0.010672,0.015046,0.009858,0.019359,0.019124
1993,-0.007379,0.006146,0.008846,0.009134,0.006758,0.006351,0.010807,0.007852,0.008616,0.021636,0.029014


### Monthly New Vars

In [27]:
analyze_result_files(monthly_new_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.021227,0.021657,0.025623,0.027445,0.028894,0.025562,0.024595,0.026396,0.022299,0.010018,-0.011209
1986,0.014041,0.010881,0.011083,0.010564,0.013008,0.017080,0.019715,0.020915,0.013741,0.014658,0.000617
1987,0.005483,0.006675,0.007595,0.007657,0.005513,-0.000242,0.009776,0.000927,-0.004270,-0.000504,-0.005987
1988,0.012986,0.015682,0.015753,0.014131,0.017599,0.011515,0.014437,0.013684,0.015808,0.010321,-0.002666
1989,0.020265,0.024614,0.022886,0.027525,0.022905,0.022971,0.017822,0.016974,0.027529,0.014364,-0.005901
1990,-0.010062,-0.004216,0.000009,-0.003305,-0.005960,-0.002602,-0.001126,-0.004369,-0.002891,0.000106,0.010168
1991,0.046924,0.034234,0.036147,0.025101,0.021917,0.020934,0.024434,0.024820,0.026222,0.026145,-0.020780
1992,0.005626,0.014594,0.005650,0.003354,0.008566,0.013060,0.006744,0.009220,0.006928,0.010292,0.004666
1993,0.004896,0.009593,0.007133,0.009657,0.012989,0.010843,0.005777,0.013943,0.009391,0.011753,0.006857


### RF results reference

In [28]:
quarterly_restricted_result_dir_RF = 'Info Processing and Mutual Funds/new_input/results/result_w_c_q_new_all_restrict_95_stocks.csv'
quarterly_restricted_result_dir_RF = 'Info Processing and Mutual Funds/new_input/results/result_w_c_q_new_all_restrict_95.csv'

In [29]:
analyze_result_files(quarterly_restricted_result_dir_RF)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1986,-0.042276,0.000715,0.029379,0.030065,0.037146,0.052625,0.051450,0.055685,0.061998,0.054150,0.096426
1987,-0.047072,-0.034725,-0.020583,-0.014037,-0.004612,0.013922,0.007724,0.015038,0.038979,0.057305,0.104377
1988,-0.032986,-0.013332,0.016615,0.027845,0.039641,0.027500,0.036896,0.029400,0.062737,0.069541,0.102527
1989,-0.039194,0.001860,0.021272,0.041653,0.046331,0.039573,0.048253,0.069729,0.081896,0.093934,0.133128
1990,-0.180627,-0.135499,-0.088966,-0.049471,-0.040899,-0.012314,-0.025692,-0.014494,-0.000844,0.008721,0.189348
1991,0.084129,0.082954,0.050934,0.059230,0.049883,0.066490,0.068891,0.083475,0.116456,0.162577,0.078448
1992,-0.041512,0.009157,0.014145,0.038007,0.030571,0.029338,0.023477,0.024085,0.005520,-0.010597,0.030915
1993,-0.015621,0.022742,0.016251,0.047191,0.018877,0.038794,0.055268,0.062425,0.061826,0.051368,0.066990
1994,-0.107817,-0.054129,-0.010377,0.004112,0.000562,0.006738,-0.000606,0.007396,-0.005335,0.043673,0.151490


In [30]:
compute_returns(quarterly_restricted_result_dir_RF)

/tmp/ipykernel_115797/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,-0.035549
1,1.0,-0.002851
2,2.0,0.012681
3,3.0,0.026528
4,4.0,0.030880
5,5.0,0.031569
6,6.0,0.039013
7,7.0,0.044643
8,8.0,0.047770
9,9.0,0.066568
